In [42]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import RFECV
import numpy as np
from sklearn.ensemble import RandomForestClassifier

import matplotlib.pyplot as plt

In [38]:
data_dir = "/Users/shehbeel/Documents/PBTA-miRNA-Analysis/data/"

df = pd.read_csv(data_dir + 'mb_pbta_mirna.csv')

In [45]:
df

,class,let-7a-2-3p,let-7a-3p,let-7a-5p,let-7b-5p,let-7c-3p,let-7c-5p,let-7d-3p,let-7d-5p,let-7e-3p,...,miR-944,miR-95-3p,miR-95-5p,miR-9-5p,miR-96-3p,miR-96-5p,miR-98-3p,miR-99a-5p,miR-99b-3p,miR-99b-5p
0,group4,0.000021,0.000396,0.623749,0.235732,0.001238,0.397037,0.002096,0.052650,0.000543,...,0.000000,0.001244,0.000000,0.867301,0.000000,0.000105,0.000003,0.467130,0.000651,0.023314
1,shh,0.000066,0.000756,0.949854,0.459211,0.000312,0.488999,0.017457,0.195213,0.001016,...,0.000005,0.008301,0.000009,0.720100,0.000000,0.000156,0.000009,0.143314,0.001020,0.022673
2,shh,0.000031,0.000266,0.360419,0.182326,0.000541,0.209765,0.009398,0.070637,0.000624,...,0.000009,0.007082,0.000002,0.265383,0.000007,0.000440,0.000012,0.128355,0.000518,0.010669
3,shh,0.000144,0.000397,1.000000,0.541517,0.000707,0.670771,0.011183,0.117447,0.000838,...,0.000009,0.003454,0.000004,0.577943,0.000013,0.000201,0.000031,0.469522,0.001262,0.021444
4,group4,0.000140,0.000124,0.551274,0.232158,0.001113,0.385674,0.002171,0.040326,0.000615,...,0.000000,0.000202,0.000000,0.461546,0.000000,0.000700,0.000010,0.466586,0.000856,0.024345
5,group3,0.000012,0.000062,0.251669,0.045793,0.000146,0.093762,0.002515,0.039105,0.000665,...,0.000000,0.000502,0.000000,0.077160,0.000497,0.546387,0.000007,0.099849,0.001061,0.035009
6,group3,0.000037,0.000098,0.396680,0.095476,0.001044,0.295310,0.002314,0.034163,0.000384,...,0.000003,0.000138,0.000007,1.000000,0.000007,0.004850,0.000007,0.564924,0.001448,0.035173
7,shh,0.000108,0.000110,0.209708,0.101648,0.000415,0.120649,0.003664,0.037256,0.000235,...,0.000007,0.000319,0.000000,0.171490,0.000000,0.000319,0.000007,0.087714,0.000293,0.007802
8,group3,0.000078,0.000323,0.729035,0.350305,0.004271,0.547715,0.011622,0.104420,0.000580,...,0.000000,0.000084,0.000000,0.674608,0.000066,0.033341,0.000030,0.620002,0.001262,0.023962
9,group4,0.000214,0.000421,0.556721,0.195960,0.000789,0.318187,0.001863,0.061527,0.000512,...,0.000000,0.000184,0.000000,1.000000,0.000000,0.003510,0.000013,0.369452,0.000489,0.019117


## Data preprocessing

In [41]:
# Split the dataset into training data and labels
X = df.loc[:, df.columns != 'class'].values
y = df.loc[:, df.columns == 'class'].values.ravel()

# Sanity check
print(X[:5])
print(y[:5])

[[2.10000000e-05 3.95820000e-04 6.23749194e-01 ... 4.67130456e-01
  6.50704000e-04 2.33143920e-02]
 [6.61000000e-05 7.55926000e-04 9.49853776e-01 ... 1.43314073e-01
  1.02050000e-03 2.26730480e-02]
 [3.06000000e-05 2.65969000e-04 3.60419055e-01 ... 1.28354921e-01
  5.17816000e-04 1.06693720e-02]
 [1.44096000e-04 3.97356000e-04 1.00000000e+00 ... 4.69521514e-01
  1.26193200e-03 2.14441040e-02]
 [1.39986000e-04 1.23708000e-04 5.51273871e-01 ... 4.66585714e-01
  8.56192000e-04 2.43445080e-02]]
['group4' 'shh' 'shh' 'shh' 'group4']


## RFE-CV

In [49]:
rfecv = RFECV(estimator=RandomForestClassifier())
model = RandomForestClassifier()

pipeline = Pipeline([('Feature Selection', rfecv), ('Model', model)])
cv = RepeatedStratifiedKFold(n_splits=2, n_repeats=5, random_state=123)
n_scores = cross_val_score(pipeline, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
np.mean(n_scores)

/Users/shehbeel/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_split.py:684: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/Users/shehbeel/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_split.py:684: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/Users/shehbeel/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_split.py:684: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/Users/shehbeel/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_split.py:684: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/Users/shehbeel/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_split.py:684: UserWarning: The least populated class in y has only 1 members, whi

KeyboardInterrupt: 

In [47]:
pipeline.fit(X, y)

/Users/shehbeel/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_split.py:684: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(


KeyboardInterrupt: 

In [46]:
print('Optimal number of features : %d' % rfecv.n_features_)

AttributeError: 'RFECV' object has no attribute 'n_features_'

In [ ]:
rfecv.support_
#rfecv_df = pd.DataFrame(rfecv.ranking_,index=X.columns,columns=['Rank']).sort_values(by='Rank',ascending=True)
rfecv_df = pd.DataFrame(rfecv.ranking_,index=features,columns=['Rank']).sort_values(by='Rank',ascending=True)

rfecv_df.head()

In [ ]:

plt.figure(figsize=(12,6))
plt.xlabel('Number of features selected')
plt.ylabel('Cross validation score (nb of correct classifications)')
plt.plot(range(1, len(rfecv.grid_scores_) + 1), rfecv.grid_scores_)
plt.show()

In [ ]:
rfecv_rank_df = pd.DataFrame(rfecv.ranking_, index=features, columns=['Rank']).sort_values(by='Rank', ascending=True)
rfecv_rank_df.head()
#rfecv_rank_df.to_csv('pbta_rfecv_selected_features.csv')

In [ ]:
rfecv_cols = ['Sample_ID', 'class'] + rfecv_rank_df[rfecv_rank_df['Rank']==1].index.to_list()
df[rfecv_cols]
#df[rfecv_cols].to_csv('pbta_rfecv_selected_features_df.csv')